# Phase 1: ASK

The business tasks in this scenario include:

1. Analyzing smart device fitness data: The junior data analyst is tasked with analyzing smart device data related to one of Bellabeat's products to gain insight into how consumers are using their smart devices. The purpose of this analysis is to identify trends and patterns that can inform Bellabeat's marketing strategy.

2. Developing marketing strategy recommendations: Based on the analysis of smart device data, the data analyst is also responsible for providing high-level recommendations for Bellabeat's marketing strategy. The recommendations should be based on insights from the data and should align with Bellabeat's overall mission and business goals.

The key stakeholders in this scenario include:

1. Urška Sršen: As Bellabeat's cofounder and Chief Creative Officer, Sršen is responsible for driving the company's growth and innovation. She has identified the need to analyze smart device data as a way to unlock new growth opportunities for the company, and has tasked the marketing analytics team with this responsibility.

2. Sando Mur: As a mathematician and Bellabeat's cofounder, Mur is a key member of the executive team. He is likely to be involved in the decision-making process related to the marketing strategy recommendations that come out of the data analysis.

3. Bellabeat marketing analytics team: This team of data analysts is responsible for collecting, analyzing, and reporting data that helps guide Bellabeat's marketing strategy. The junior data analyst who is the main character in this scenario is a member of this team.

4. Bellabeat customers: The insights and recommendations generated from the smart device data analysis and marketing strategy development will ultimately impact Bellabeat's customers. The goal is to use the data to create marketing strategies that better meet their needs and preferences.

# Phase 2: Prepare

**Determine the credibility of the data.**

It is stored in CSV files and organized in long format. It is rated 10/10 for usability on Kaggle, and the original data source can be located and cited using a DOI. The data appears to be comprehensive and updated regularly. However, it is unclear whether there are any issues with bias in the data. Licensing, privacy, security, and accessibility are addressed by examining the original data source. The data's integrity is assumed to be valid as it has a license, acknowledgement for the authors, and the consent from the users

# Import neccessary library

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Loading the dataset

In [ ]:
import pandas as pd
import os

# Set the path to the input folder in Kaggle
path = "../input/fitbit/Fitabase Data 4.12.16-5.12.16"

# Get a list of all files in the input folder
all_files = os.listdir(path)

# Select only CSV files
csv_files = [file for file in all_files if file.endswith(".csv")]

# Group the CSV files by their filename
daily_files = [file for file in csv_files if "daily" in file.lower()]
hourly_files = [file for file in csv_files if "hourly" in file.lower()]
minute_files = [file for file in csv_files if "minute" in file.lower()]
other_files = [file for file in csv_files if file not in daily_files and file not in hourly_files and file not in minute_files]

# Create empty dictionaries to store the dataframes
daily_dfs = {}
hourly_dfs = {}
minute_dfs = {}
other_dfs = {}

# Store the daily files into the "daily_dfs" dictionary
for file in daily_files:
    filename, file_extension = os.path.splitext(file)
    df = pd.read_csv(os.path.join(path, file))
    daily_dfs[filename] = df

# Store the hourly files into the "hourly_dfs" dictionary
for file in hourly_files:
    filename, file_extension = os.path.splitext(file)
    df = pd.read_csv(os.path.join(path, file))
    hourly_dfs[filename] = df

# Store the minute files into the "minute_dfs" dictionary
for file in minute_files:
    filename, file_extension = os.path.splitext(file)
    df = pd.read_csv(os.path.join(path, file))
    minute_dfs[filename] = df

# Store the other files into the "other_dfs" dictionary
for file in other_files:
    filename, file_extension = os.path.splitext(file)
    df = pd.read_csv(os.path.join(path, file))
    other_dfs[filename] = df

# Group all dataframes into a list
data_frames = [daily_dfs, hourly_dfs, minute_dfs, other_dfs]


# Phase 3: Process

# Data cleaning

## Step 1: Handling missing value (remove/fill)

In [ ]:
# Check for missing values
for df_dict in data_frames:
    for filename, df in df_dict.items():
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(filename)
        print(df.isnull().sum())

In general, the data is relatively complete. There is not many missing values except for the 'weightLogInfo_merged' data. There are 65 rows with missing value regardless of 'Fat' column. That is almost 100% missing (65/67). Hence there is no point of keeping this collumn so I am going to remove it. Also I will choose to remove missing values in this project instead of filling in missing values because there is not many missing values so when we remove rows with missing values we still have enough data to answer the busines objectives.

In [ ]:
# Drop the 'Fat' column of the weightLogInfo_merged dataset
#because there are only two rows contain value in that column
other_dfs['weightLogInfo_merged'].drop(columns=["Fat"], inplace=True, axis=1)


In [ ]:
#Remove missing values
for df_dict in data_frames:
    for filename, df in df_dict.items():
        df.dropna(inplace=True)

## Step 2: Remove any duplicates

In [ ]:
for df_dict in data_frames:
    for filename, df in df_dict.items():
        df.drop_duplicates(inplace=True)
        
        #Reset the index
        df.reset_index(drop=True, inplace=True)
        

## Step 3: Format string data (if applicable)

Taking a look at the dataset, there is no columns with string data so this step is not needed. 

## Step 4: Handling outliers (remove/handle)

In [ ]:
# determine outlier
for df_dict in data_frames:
    for filename, df in df_dict.items():
        #Exclude dataset contains "boolean" data
        if filename == "weightLogInfo_merged":
            continue
        Q1 = df.quantile(0.25)
        Q3 = df.quantile(0.75)
        IQR = Q3 - Q1
        
        #Remove outliers
        df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]


It is common for some individuals who are highly active exists in the dataset, so I choose remove them from the dataset to avoid their data to skew the dataset too much. However, it is best practice to investigate each outlier before deciding whether to remove or standardise them because outliers data may contain useful phenomeno.

## Step 5: Convert data types (if applicable)

There is no need for this step in this project

# Data summary and manipulation

### Summary

After investigate the data, I recognise the datasets related to physcial activity are broken down into daily, hourly, minute. However, all of them have been merged into 'dailyActivity_merged.csv' file.  I will only use that file to find the insights for physical activities since it is enough to complete the business goals.

In [ ]:
# Perform data summary for each of the pandas DataFrames
for df_dict in data_frames:
    for filename, df in df_dict.items():
        if filename in ['dailyActivity_merged', 'sleepDay_merged', 'heartrate_seconds_merged', 'weightLogInfo_merged']:
            print("=================================================================================")
            print(f"Data summary for file {filename}:")
            # Count the number of unique values in the "Id" column
            unique_id_count = df["Id"].nunique()

            # Print the result
            print(f"The number of unique Id values is: {unique_id_count}")
            print(df.info())
            print(df.describe())
            print("\n")


From the summary above we can observer the following:
- 33 people provide data for physical activities
- 24 people provide data for sleep 
- 14 people provide data for heart rate
- 8 people provide data for weight

### Manipulation

I will add two more extra collumn to the 'daily_activity_merged' dataset called 'weekday', showing what day of the week for the current day, and 'TotalActiveMinutes', which sum up active minutes of the given day.

### Daily activity

In [ ]:
daily_activity_df = daily_dfs['dailyActivity_merged']
# Create a new column with the weekdays
daily_activity_df['ActivityDate'] = pd.to_datetime(daily_activity_df["ActivityDate"])
daily_activity_df['weekday'] = daily_activity_df['ActivityDate'].dt.day_name()
daily_activity_df['TotalActiveMinutes'] = daily_activity_df['FairlyActiveMinutes'] + daily_activity_df['LightlyActiveMinutes'] + daily_activity_df['VeryActiveMinutes']

# Rename the columns
daily_activity_df = daily_activity_df.rename(columns={'ActivityDate': 'date'})

daily_activity_df['date'] = pd.to_datetime(daily_activity_df["date"])



daily_activity_df.to_csv('daily_activity_updated.csv', index=False)


In [ ]:
daily_activity_df.head()

### Heart Rate

For the heart rate data, I will separate 'Time' column into 2 different columns named 'date', contains date information, and 'time', contain time information. Also, I create an extra column name 'Time_Category' to classify time of the day into 4 different group: morning (5am to 12pm), afternoon(12pm to 17pm), evening(17pm to 12am), and night (12am to 5am).

In [ ]:
heart_rate_df = other_dfs["heartrate_seconds_merged"] 
heart_rate_df['Time'] = pd.to_datetime(heart_rate_df["Time"])
# Create a new column with the weekdays
heart_rate_df['weekday'] = heart_rate_df['Time'].dt.day_name()
# Extract the date and time information into separate columns
heart_rate_df['date'] = heart_rate_df['Time'].dt.date
heart_rate_df['time'] = heart_rate_df['Time'].dt.time
heart_rate_df = heart_rate_df.drop('Time', axis=1)

# Define a function to classify time into categories
def classify_time(t):
    hour = t.hour
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'


# Apply the function to the 'Time' column to create a new column 'Time_Category'
heart_rate_df['Time_Category'] = heart_rate_df['time'].apply(classify_time)

#Export to new csv file
heart_rate_df.to_csv('hear_rate_updated.csv', index=False)

# Show the resulting dataframe
heart_rate_df.head()

After that, I compute the average heart of people and group them by Id and weekday for later analysis

In [ ]:
# group the dataframe by 'Id' and 'weekday' and compute the mean of 'Value'
heart_rate_mean = heart_rate_df.groupby(['Id', 'weekday','date'])['Value'].mean()

# reset the index of the resulting dataframe to make 'Id' and 'weekday' columns again
heart_rate_mean = heart_rate_mean.reset_index()

# Rename the columns
heart_rate_mean = heart_rate_mean.rename(columns={'Value': 'AvgHeartRate'})

# print the resulting dataframe
heart_rate_mean.head()

### Sleep

Similarly, for sleep datset, I breakdown the 'SleepDay' column in to 'date' and 'time'. Then I added an additional column called 'weekday' to classify what day of week is that for the date.

In [ ]:
sleep_df = other_dfs['sleepDay_merged']
sleep_df['SleepDay'] = pd.to_datetime(sleep_df["SleepDay"])
# Extract the date and time information into separate columns
sleep_df['date'] = sleep_df['SleepDay'].dt.date
sleep_df['time'] = sleep_df['SleepDay'].dt.time

# Create a new column with the weekdays
sleep_df['weekday'] = sleep_df['SleepDay'].dt.day_name()

#Drop the SleepDay column
sleep_df = sleep_df.drop('SleepDay', axis=1)

#Export to new csv file
sleep_df.to_csv('sleep_updated.csv', index=False)

# sleep_df = sleep_df.drop('Time', axis=1)
sleep_df.head()

I compute the total minutes a sleep and group the by Id and weekday for later analysis

In [ ]:
# group the dataframe by 'Id', 'weekday', and 'date', and compute the mean of 'TotalMinutesSleep' and 'TotalTimeInBed'
sleep_mean = sleep_df.groupby(['Id', 'weekday', 'date'])[['TotalMinutesAsleep', 'TotalTimeInBed']].mean()

# reset the index of the resulting dataframe to make 'Id', 'weekday', and 'date' columns again
sleep_mean = sleep_mean.reset_index()

# print the resulting dataframe
sleep_mean.head()

### Merge activity, heart rate, sleep

In [ ]:
# Convert the 'date' column to datetime data type
daily_activity_df['date'] = pd.to_datetime(daily_activity_df['date'])
heart_rate_mean['date'] = pd.to_datetime(heart_rate_mean['date'])
sleep_mean['date'] = pd.to_datetime(sleep_mean['date'])

# Merge dataframes on 'Id' and 'date' columns
df_merged = pd.merge(daily_activity_df, heart_rate_mean, on=['Id', 'date'])
df_merged = pd.merge(df_merged, sleep_mean, on=['Id', 'date'])
df_merged = df_merged.drop('weekday_x', axis=1)
df_merged = df_merged.drop('weekday_y', axis=1)

#Export to new csv file
df_merged.to_csv('master_merge.csv', index=False)

# Print the merged dataframe
df_merged.head(10)

## Data visualisation & Analysis

### Physical activity

In [ ]:
# Group the data by weekday and calculate the mean calories burned for each weekday
avg_calories = daily_activity_df.groupby('weekday')['Calories'].mean()

# Sort the value by descending order
avg_calories = avg_calories.sort_values()

# Create a bar graph using Pandas plot function
ax = avg_calories.plot(kind='barh',  figsize = (10,6), color='lightblue', alpha=0.7)

# Set the title and axis labels
ax.set_title('Average Calories Burned by Day of Week', fontsize=14, fontweight='bold')
ax.set_xlabel('Average Calories Burned', fontsize=12)
ax.set_ylabel('Weekday', fontsize=12)

# Add grid lines and remove spines
ax.grid(axis='x', linestyle='--', alpha=0.9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(False)

# Add labels to the bars
for i, v in enumerate(avg_calories):
    ax.text(v + 10, i - 0.1, str(int(v)), color='black', fontsize=10)
    
# save the plot to a file
plt.savefig('avg_cal_burned_by_day.png')

# Show the plot
plt.show()

In [ ]:
# Set the weekday column as a categorical data type with the desired order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_activity_df['weekday'] = pd.Categorical(daily_activity_df['weekday'], categories=weekday_order, ordered=True)

# Group the data by weekday and calculate the mean total steps for each weekday
avg_step_by_day = daily_activity_df.groupby('weekday')['TotalSteps'].mean()

# Sort the value by descending order
avg_step_by_day = avg_step_by_day.sort_values()

# Create a bar graph using Pandas plot function
ax = avg_step_by_day.plot(kind='barh',  figsize = (10,6), color='lightblue', alpha=0.7)

# Set the title and axis labels
ax.set_title('Average Total Steps by Day of Week', fontsize=14, fontweight='bold')
ax.set_xlabel('Average Total Steps', fontsize=12)
ax.set_ylabel('Weekday', fontsize=12)

# Add grid lines and remove spines
ax.grid(axis='x', linestyle='--', alpha=0.9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(False)

# Add labels to the bars
for i, v in enumerate(avg_step_by_day):
    ax.text(v + 100, i - 0.1, str(int(v)), color='black', fontsize=10)

# save the plot to a file
plt.savefig('avg_step_by_day.png')

# Show the plot
plt.show()

In [ ]:
# Group the data by weekday and calculate the mean total distance for each weekday
avg_total_dist_by_day = daily_activity_df.groupby('weekday')['TotalDistance'].mean()

# Sort the value by descending order
avg_total_dist_by_day = avg_total_dist_by_day.sort_values()

# Create a bar graph using Pandas plot function
ax = avg_total_dist_by_day.plot(kind='barh',  figsize = (10,6), color='lightblue', alpha=0.7)

# Set the title and axis labels
ax.set_title('Average Total Distance Traveled by Day of Week', fontsize=14, fontweight='bold')
ax.set_xlabel('Average Total Distance (km)', fontsize=12)
ax.set_ylabel('Weekday', fontsize=12)

# Add grid lines and remove spines
ax.grid(axis='x', linestyle='--', alpha=0.9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(False)

# Add labels to the bars
for i, v in enumerate(avg_total_dist_by_day):
    ax.text(v + 0.3, i - 0.1, '{:.1f}'.format(v), color='black', fontsize=10)

# save the plot to a file    
plt.savefig('avg_dist_by_day.png')

# Show the plot
plt.show()

In [ ]:
# Set the weekday column as a categorical data type with the desired order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_activity_df['weekday'] = pd.Categorical(daily_activity_df['weekday'], categories=weekday_order, ordered=True)

# Group the data by weekday and calculate the sum values for each column
avg_activity_by_day = daily_activity_df.groupby('weekday').mean()

# Sort the value by total active minutes in descending order descending order
avg_activity_by_day = avg_activity_by_day.sort_values(by='TotalActiveMinutes')

# Set the colors for the bars
colors = ['#4b7bec', '#a55eea', '#f53b57', '#ced6e0', '#ff6b81', '#576574']

# Create a bar graph using Pandas plot function
ax = avg_activity_by_day.plot(kind='barh', y=['TotalActiveMinutes', 'SedentaryMinutes'], figsize = (12,6), color=colors,alpha=0.7)

# Set the title and axis labels
ax.set_title('Average Active Minutes vs Sedentary Minutes by Day of Week', fontsize=14, fontweight='bold')
ax.set_xlabel('Minutes', fontsize=12)
ax.set_ylabel('Weekday', fontsize=12)

# Set the legend
ax.legend(['Active', 'Sedentary'], bbox_to_anchor=(1, 1))



# Add grid lines and remove spines
ax.grid(axis='y', linestyle='--', alpha=0.9)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# save the plot to a file
plt.savefig('avg_active_minutes_vs_sedentary_by_day.png')

# Show the plot
plt.show()

In [ ]:
# Set the weekday column as a categorical data type with the desired order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_activity_df['weekday'] = pd.Categorical(daily_activity_df['weekday'], categories=weekday_order, ordered=True)

# Set the colors for the bars
colors = ['#6ab04c', '#2c3e50', '#3498db']

# Group the data by weekday and calculate the sum values for each column
avg_distance_by_day = daily_activity_df.groupby('weekday').mean()

# Sort the value by total light active in descending order descending order
avg_distance_by_day = avg_distance_by_day.sort_values(by='LightActiveDistance')

# Create a bar graph using Pandas plot function
ax = avg_distance_by_day.plot(kind='barh', y=["VeryActiveDistance","ModeratelyActiveDistance","LightActiveDistance"], figsize=(12, 6), width=0.8, color=colors, alpha=0.8)

# Set the title and axis labels
ax.set_title('Avereage Active Distance by Day of Week', fontsize=14, fontweight='bold')
ax.set_xlabel('Miles', fontsize=12)
ax.set_ylabel('Weekday', fontsize=12)


# Add horizontal grid lines
ax.grid(axis='x', linestyle='--', linewidth=0.5, alpha=0.5)


# Set the legend
ax.legend(['Very Active', 'Moderate Active', 'Light Active'],
          bbox_to_anchor=(1, 1))

# save the plot to a file
plt.savefig('avg_active_distance_by_day.png')

# Display the plot
plt.show()

In [ ]:
# Set the weekday column as a categorical data type with the desired order
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
daily_activity_df['weekday'] = pd.Categorical(daily_activity_df['weekday'], categories=weekday_order, ordered=True)

# Set the colors for the bars
colors = ['#6ab04c', '#2c3e50', '#3498db']

# Group the data by weekday and calculate the sum values for each column
avg_active_minutes_by_day = daily_activity_df.groupby('weekday').mean()

# Sort the value by total light active minutes in descending order descending order
avg_active_minutes_by_day = avg_active_minutes_by_day.sort_values(by='LightlyActiveMinutes')

# Create a bar graph using Pandas plot function
ax = avg_active_minutes_by_day.plot(kind='barh', y=['VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes' ], figsize=(12, 6), width=0.8, color=colors, alpha=0.8)

# Set the title and axis labels
ax.set_title('Avereage Active Minutes by Level', fontsize=14, fontweight='bold')
ax.set_xlabel('Minutes', fontsize=12)
ax.set_ylabel('Weekday', fontsize=12)


# Add horizontal grid lines
ax.grid(axis='x', linestyle='--', linewidth=0.5, alpha=0.5)


# Set the legend
ax.legend(['Very Active', 'Fairly Active', 'Lightly Active'], bbox_to_anchor=(1, 1))


# save the plot to a file
plt.savefig('avg_active_minutes_by_level.png')

# Display the plot
plt.show()

### Heart Rate

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Compute the average heart rate value using "Value" column
hr_avg = heart_rate_df.groupby(["weekday", "Time_Category"])["Value"].mean().reset_index()

# Sort the data by the average value of heart rate for "Time_category" and then by "weekday"
hr_avg = hr_avg.sort_values(["Time_Category", "Value"], ascending=[True, False])

# Create a horizontal bar chart
fig, ax = plt.subplots(figsize=(10, 8))
sns.barplot(x="Value", y="weekday", hue="Time_Category", data=hr_avg, ax=ax, orient="h", palette='husl')

# Set the title and axes labels
ax.set_title("Average Heart Rate by Day of Week and Time of Day")
ax.set_xlabel("Average Heart Rate")
ax.set_ylabel("Weekday")

# Move the legend to the top right corner
ax.legend(loc="upper right", bbox_to_anchor=(1.2, 1))

# save the plot to a file
plt.savefig('avg_heart_rate_by_day_and_daytime.png')

# Display the plot
plt.show()


### Sleep

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Compute the average of "TotalMinutesAsleep", group it by 'weekday', and sort it in descending order
sleep_grouped = sleep_df.groupby('weekday')['TotalMinutesAsleep'].mean().sort_values(ascending=False)

# Create a box plot
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x='weekday', y='TotalMinutesAsleep', data=sleep_df, order=sleep_grouped.index, showfliers=False, ax=ax)

plt.title('Average TotalMinutesAsleep by Day of Week')
plt.xlabel('Weekday')
plt.ylabel('TotalMinutesAsleep')

# save the plot to a file
plt.savefig('avg_sleep_by_day.png')

### Examine the correlation between variables

In [ ]:
# Correlation between physcial activities variables
# Correlation matrix
corr = daily_activity_df.corr()

# Plot correlation matrix
plt.figure(figsize=(12,10))
plt.title("Daily Activity")
mask = np.triu(np.ones_like(daily_activity_df.corr(), dtype=bool))
sns.heatmap(daily_activity_df.corr(), mask=mask, cmap='coolwarm', annot=True)

# save the plot to a file
plt.savefig('activity_corr_heat_map.png')

#display the heatmap
plt.show()



In [ ]:
# Correlation between physcial, heart rate, and sleep variable
# Correlation matrix
corr = df_merged.corr()

# Plot correlation matrix
plt.figure(figsize=(12,10))
plt.title("Daily Activity")
mask = np.triu(np.ones_like(df_merged.corr(), dtype=bool))
sns.heatmap(df_merged.corr(), mask=mask, cmap='coolwarm', annot=True)

# save the plot to a file
plt.savefig('activity_heartrate_sleep_heatmap.png')

#Display the heat map
plt.show()



# Phase 4: Anayse

### Physical activity

Fitbit trackers measure physical activity in terms of "active minutes," which are the minutes during which you engage in moderate to intense physical activity.

Fitbit defines three levels of physical activity:

1. Lightly Active Minutes: These are minutes when you are active, but not necessarily exercising. This might include activities like walking around the house, doing light chores, or standing up from your desk periodically.

2. Fairly Active Minutes: These are minutes when you are engaging in moderate-intensity exercise or activity. This might include activities like brisk walking, cycling, or doing household chores that require a bit more effort.

3. Very Active Minutes: These are minutes when you are engaging in high-intensity exercise or activity. This might include activities like running, hiking, or playing a sport at a competitive level.

Fitbit uses its sensors to track your movements and heart rate to determine the level of activity you are engaged in, and then assigns active minutes accordingly. The more active you are, the more active minutes you will accumulate.

<div style="display:flex">
    <div style="flex:1;padding-right:5px;">
        <img src="avg_active_minutes_vs_sedentary_by_day.png">
    </div>
    <div style="flex:1;padding-left:5px;">
        <img src="avg_active_minutes_by_level.png">
    </div>
</div>


On average, Saturday, Tuesday, and Friday are the top three most active days of the week, with Monday, Wednesday, Thursday, and Sunday following suit. People typically accumulate around 20 minutes of very active minutes and 15 minutes of fairly active minutes per day. However, the level of lightly active minutes varies across the days, with Saturday, Friday, and Tuesday being the top three, followed by Monday, Wednesday, Thursday, and Sunday.

It is noteworthy that the level of lightly active minutes is a significant factor that distinguishes the most active day from the least active day of the week. These findings suggest that people who engage in more light activities throughout the day tend to be more active overall.

![avg_cal_burned_by_day](avg_cal_burned_by_day.png)

In terms of calorie expenditure, it appears that Tuesday, Saturday, and Friday are the top three days where people tend to burn the most calories, followed by Monday, Wednesday, Sunday, and Thursday. This pattern is similar to the distribution of active minutes throughout the week, indicating that there may be a relationship between physical activity and calorie burn. These insights could be valuable in developing strategies to encourage individuals to be more physically active and burn more calories, potentially leading to improved health outcomes.

![avg_dist_by_day](avg_dist_by_day.png)

![avg_step_by_day](avg_step_by_day.png)

Upon analyzing the total distance traveled and total steps taken by day of the week, it becomes evident that Saturday, Tuesday, and Friday are the top three most active days. Interestingly, outdoor activities seem to be more common on Saturday and Tuesday, while indoor activities are more prevalent on Friday. This is reflected in the average total distance traveled and average total steps taken, with Saturday and Tuesday consistently ranking within the top three, while Friday falls to the bottom three in these categories.

Given that the majority of active minutes are accounted for by light activities, it is reasonable to infer that people engage in outdoor light exercises such as jogging on Saturday and Tuesday, whereas on Friday, indoor activities are more prevalent. These insights can be useful in developing targeted interventions and promoting physical activity by emphasizing outdoor activities as a means of staying active and healthy

### Heart rate

![avg_heart_rate_by_day_and_daytime](avg_heart_rate_by_day_and_daytime.png)

Through an analysis of the average heart rate by day of the week and time of day, we can gain insights into when people tend to engage in high-intensity exercise. Among the top three most active days, we observe that people tend to perform high-intensity exercises in the afternoon on Saturday, and in the evening or afternoon on Tuesday. On Friday, high-intensity exercise is more commonly performed in the evening.

These findings provide valuable information for fitness product manufacturers and wellness professionals to create personalized recommendations that align with users' schedules and natural tendencies. For instance, they can suggest that people engage in high-intensity exercise during specific times of the day to maximize the benefits of their workout, based on the patterns observed in the data. Furthermore, these insights can inform the design and marketing of fitness products to better meet the needs of consumers, ultimately leading to improved health and wellness outcomes.

### Sleep

![average_sleep](avg_sleep_by_day.png)

The box plot analysis indicates that people tend to have less total sleep time on their most active days, with Saturday, Friday, and Tuesday ranking among the bottom four. Conversely, on the least active days such as Sunday, Wednesday, and Monday, people tend to have more total minutes of sleep on average.

These insights can be used to develop personalized recommendations for improving sleep habits and overall wellness. For example, on more active days, people may benefit from incorporating activities that promote relaxation and stress reduction to help them wind down before bed. Similarly, on less active days, people may want to consider more physically demanding activities that promote better sleep quality. Additionally, fitness product manufacturers and wellness professionals can use this information to design products that integrate sleep tracking and provide personalized recommendations to users to optimize their sleep and overall health.

## Summary:

* Saturday, Tuesday, and Friday are the top three most active days, while Sunday, Wednesday, and Monday are the least active.

* People accumulate around 20 minutes of very active minutes and 15 minutes of fairly active minutes per day.
 
* Lightly active minutes vary across the days and are a significant factor in distinguishing the most active day from the least active day.

* Tuesday, Saturday, and Friday are the top three days where people tend to burn the most calories, followed by Monday, Wednesday, Sunday, and Thursday.

* Saturday, Tuesday, and Friday are the top three most active days in terms of total distance traveled and total steps taken.

* Outdoor activities seem to be more common on Saturday and Tuesday, while indoor activities are more prevalent on Friday.

* People tend to perform high-intensity exercises in the afternoon on Saturday and in the evening or afternoon on Tuesday, with high-intensity exercise more commonly performed in the evening on Friday.

* People tend to have less total sleep time on their most active days, with Saturday, Friday, and Tuesday ranking among the bottom four.



# Phase 5: Share

## Recommendation for Bellabeat

Based on the insights provided, Bellabeat can develop a marketing strategy for one of their products as follows:

Product: Bellabeat fitness tracker

Target audience: People who are interested in improving their physical activity levels and overall wellness

Marketing Strategy:

1. Promote outdoor activities: Given that outdoor activities are more prevalent on Saturday and Tuesday, Bellabeat can emphasize the benefits of outdoor exercise in its marketing campaigns. For instance, Bellabeat can create social media campaigns featuring pictures of people engaging in outdoor activities, and highlight the health benefits of jogging, hiking, and other outdoor activities.

2. Emphasize the importance of light activities: Bellabeat can use the insight that light activities contribute significantly to overall physical activity levels in its marketing campaigns. The campaigns can focus on how incorporating light activities throughout the day can lead to improved health outcomes.

3. Personalized workout recommendations: Bellabeat can create personalized workout recommendations based on when users tend to engage in high-intensity exercises. For example, if a user tends to engage in high-intensity exercise in the afternoon on Saturdays, Bellabeat can recommend specific high-intensity exercises for that time of day.

4. Improve sleep habits: Bellabeat can use the insight that people tend to have less sleep time on their most active days to develop personalized recommendations for improving sleep habits. Bellabeat can suggest activities that promote relaxation and stress reduction to help users wind down before bed on more active days.

5. Highlight the relationship between physical activity and calorie burn: Bellabeat can emphasize the relationship between physical activity and calorie burn in its marketing campaigns. The campaigns can highlight how being physically active on specific days of the week can lead to increased calorie burn, and ultimately, improved health outcomes.

6. Use social media influencers: Bellabeat can collaborate with social media influencers who are interested in health and wellness to promote its fitness tracker. The influencers can create posts featuring the product and highlight its features and benefits.

Overall, Bellabeat can use the insights provided to develop targeted marketing campaigns that resonate with its target audience and ultimately lead to improved health and wellness outcomes.